In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from pandas import HDFStore

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    
path_to_hdf_datastore = cfg['path_to_hdf_datastore']
path_to_clean_hdf_datastore = cfg['path_to_clean_hdf_datastore']

In [2]:
# Read Data
product_assignment = pd.read_hdf(path_to_hdf_datastore, cfg['product_assignment_file'])

df = product_assignment
del product_assignment

df.head()

,TERRITORYNUMBER,TERRITORYNAME,OWNERNAME,OWNERID,BU,ASSIGNTOTERRITORYNUMBER,ASSIGNTOTERRITORYNAME,ASSIGNTOTERRITORYOWNERNAME,ASSIGNTOTERRITORYOWNERID,PRODUCT_LEVEL_1,PRODUCT_LEVEL_2
0,1016,MD,Gino Ussi,USSIG,RSS,1016,MD,Gino Ussi,USSIG,nan,nan
1,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,RESEARCH INTELLIGENCE,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,Research Intelligence,Pure
2,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,RESEARCH INTELLIGENCE,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,Research Intelligence,Research Data Management
3,105683,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,RESEARCH INTELLIGENCE,105683,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,nan,nan
4,106705,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,RESEARCH INTELLIGENCE,106705,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,nan,nan


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown'], np.nan, inplace=True)
df.replace('.*\?.*', np.nan, regex=True, inplace=True)
df

,TERRITORYNUMBER,TERRITORYNAME,OWNERNAME,OWNERID,BU,ASSIGNTOTERRITORYNUMBER,ASSIGNTOTERRITORYNAME,ASSIGNTOTERRITORYOWNERNAME,ASSIGNTOTERRITORYOWNERID,PRODUCT_LEVEL_1,PRODUCT_LEVEL_2
0,1016,MD,Gino Ussi,USSIG,RSS,1016,MD,Gino Ussi,USSIG,NaN,NaN
1,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,RESEARCH INTELLIGENCE,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,Research Intelligence,Pure
2,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,RESEARCH INTELLIGENCE,105667,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,Research Intelligence,Research Data Management
3,105683,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,RESEARCH INTELLIGENCE,105683,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,NaN,NaN
4,106705,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,RESEARCH INTELLIGENCE,106705,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
45727,122718,CCSD-CP-APAC-JAPAN-1,Katsunori Motosaka,MOTOSAKAK,CP-APAC,1016,MD,Gino Ussi,USSIG,Journals,Journals Other
45728,122718,CCSD-CP-APAC-JAPAN-1,Katsunori Motosaka,MOTOSAKAK,CP-APAC,1016,MD,Gino Ussi,USSIG,Journals,Journals Legacy
45729,122718,CCSD-CP-APAC-JAPAN-1,Katsunori Motosaka,MOTOSAKAK,CP-APAC,1016,MD,Gino Ussi,USSIG,Books,Books - Legacy
45730,122718,CCSD-CP-APAC-JAPAN-1,Katsunori Motosaka,MOTOSAKAK,CP-APAC,1016,MD,Gino Ussi,USSIG,Journals,Journals Transactions


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

124202    320
116687    192
54943     192
1825      160
1604      160
         ... 
1265        2
1258        2
1259        2
1016        1
2           1
Name: TERRITORYNUMBER, Length: 998, dtype: int64
CC-CP-LS-KA-CHINA-1           512
CC-PS-RM-JPKRTW-JP            445
SSM-PS-RM-EUROPE-3            436
SSM-PS-RM-JPKRTW-3            423
SD-A&G-APAC-INSIDE-OVERLAP    416
                             ... 
SD-A&G-EMLA                     2
SD-PS-BOOKS                     2
SD-PS-RM                        2
MD                              1
Global Sales                    1
Name: TERRITORYNAME, Length: 557, dtype: int64
Ryan Sun             512
Ryo Sato             445
Gunther Hansen       436
Carol Cheng          352
Wayne Wang           352
                    ... 
Amy Hueber             2
Valerie Thiel          2
Salvatore Gelardi      2
Kumsal Bayazit         1
Gino Ussi              1
Name: OWNERNAME, Length: 598, dtype: int64
SUNR001     512
SATOR       445
HANSENG     436
WANGW123  

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
10,PRODUCT_LEVEL_2,"[[nan, Pure, Research Data Management, Journal...",40,object,0.96
9,PRODUCT_LEVEL_1,"[[nan, Research Intelligence, Journals, Books,...",9,object,0.95
0,TERRITORYNUMBER,"[[1016, 105667, 105683, 106705, 107620, 107631...",998,int64,0.00
1,TERRITORYNAME,"[[MD, RSSD-PS-RM-PURE-HS, SSM-PS-RM-BPRESS-NOA...",557,object,0.00
2,OWNERNAME,"[[Gino Ussi, Kelechi Okere - RSSD-PS-RM-PURE-H...",598,object,0.00
3,OWNERID,"[[USSIG, OKEREK - RSSD-PS-RM-PURE-HS - 105667,...",598,object,0.00
4,BU,"[[RSS, RESEARCH INTELLIGENCE, A&G-MAEU, A&G-NO...",16,object,0.00
5,ASSIGNTOTERRITORYNUMBER,"[[1016, 105667, 105683, 106705, 107620, 107631...",998,int64,0.00
6,ASSIGNTOTERRITORYNAME,"[[MD, RSSD-PS-RM-PURE-HS, SSM-PS-RM-BPRESS-NOA...",557,object,0.00
7,ASSIGNTOTERRITORYOWNERNAME,"[[Gino Ussi, Kelechi Okere - RSSD-PS-RM-PURE-H...",598,object,0.00


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove other columns
df = df.drop(columns=[])

-------------------- Dropping Columns with missing data --------------------
Following 0 columns have missing data over the threshold and will be removed
Index([], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
10,PRODUCT_LEVEL_2,"[[nan, Pure, Research Data Management, Journal...",40,object,0.96
9,PRODUCT_LEVEL_1,"[[nan, Research Intelligence, Journals, Books,...",9,object,0.95
0,TERRITORYNUMBER,"[[1016, 105667, 105683, 106705, 107620, 107631...",998,int64,0.00
1,TERRITORYNAME,"[[MD, RSSD-PS-RM-PURE-HS, SSM-PS-RM-BPRESS-NOA...",557,object,0.00
2,OWNERNAME,"[[Gino Ussi, Kelechi Okere - RSSD-PS-RM-PURE-H...",598,object,0.00
3,OWNERID,"[[USSIG, OKEREK - RSSD-PS-RM-PURE-HS - 105667,...",598,object,0.00
4,BU,"[[RSS, RESEARCH INTELLIGENCE, A&G-MAEU, A&G-NO...",16,object,0.00
5,ASSIGNTOTERRITORYNUMBER,"[[1016, 105667, 105683, 106705, 107620, 107631...",998,int64,0.00
6,ASSIGNTOTERRITORYNAME,"[[MD, RSSD-PS-RM-PURE-HS, SSM-PS-RM-BPRESS-NOA...",557,object,0.00
7,ASSIGNTOTERRITORYOWNERNAME,"[[Gino Ussi, Kelechi Okere - RSSD-PS-RM-PURE-H...",598,object,0.00


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

In this case nothing needs imputing

In [8]:
#Filling all missing values with 0
# data = data.fillna(0)

#Filling missing values with the most frequest values

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['ASSIGNTOTERRITORYNUMBER', 'TERRITORYNUMBER'])

# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
9,PRODUCT_LEVEL_1,"[[nan, Research Intelligence, Journals, Books,...",9,object,0.95
4,BU,"[[RSS, RESEARCH INTELLIGENCE, A&G-MAEU, A&G-NO...",16,object,0.00
10,PRODUCT_LEVEL_2,"[[nan, Pure, Research Data Management, Journal...",40,object,0.96


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
9,PRODUCT_LEVEL_1,"[[nan, Research Intelligence, Journals, Books,...",9,object,0.95
4,BU,"[[Other, RESEARCH INTELLIGENCE, A&G-MAEU, A&G-...",14,object,0.00
10,PRODUCT_LEVEL_2,"[[nan, Pure, Research Data Management, Journal...",39,object,0.96


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 2 features - **********************
 ASSIGNTOTERRITORYNUMBER standardized to ASSIGNTOTERRITORYNUMBER
 TERRITORYNUMBER standardized to TERRITORYNUMBER


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
print(columns_to_bin)

['PRODUCT_LEVEL_1', 'BU', 'PRODUCT_LEVEL_2']


In [18]:
# Specify Columns to encode
columns_to_exclude = []
columns_to_include = columns_to_bin
columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
columns_to_encode

['PRODUCT_LEVEL_1', 'BU', 'PRODUCT_LEVEL_2']

In [19]:
for column in columns_to_encode:
    encoded_columns = pd.get_dummies(df[column])
    print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
    df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

Encoding collumns : PRODUCT_LEVEL_1 to 8 new encoded columns
Encoding collumns : BU to 14 new encoded columns
Encoding collumns : PRODUCT_LEVEL_2 to 38 new encoded columns


In [20]:
df.shape

(45732, 68)

In [21]:
df.head()

,TERRITORYNUMBER,TERRITORYNAME,OWNERNAME,OWNERID,ASSIGNTOTERRITORYNUMBER,ASSIGNTOTERRITORYNAME,ASSIGNTOTERRITORYOWNERNAME,ASSIGNTOTERRITORYOWNERID,Books,Engineering,...,Reaxys,Reaxys Medicinal Chemistry,Research Data Management,SCOPUS,SSRN,SciVal Tools,Scopus Custom Data,bepress,eBooks,embase
0,-0.82,MD,Gino Ussi,USSIG,-0.45,MD,Gino Ussi,USSIG,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.27,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,2.55,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.27,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,2.55,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1.27,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,2.55,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.29,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,2.58,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,0,0,...,0,0,0,0,0,0,0,0,0,0


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = []

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calculate new variable activity_length

In [25]:
 df.head()

,TERRITORYNUMBER,TERRITORYNAME,OWNERNAME,OWNERID,ASSIGNTOTERRITORYNUMBER,ASSIGNTOTERRITORYNAME,ASSIGNTOTERRITORYOWNERNAME,ASSIGNTOTERRITORYOWNERID,Books,Engineering,...,Reaxys,Reaxys Medicinal Chemistry,Research Data Management,SCOPUS,SSRN,SciVal Tools,Scopus Custom Data,bepress,eBooks,embase
0,-0.82,MD,Gino Ussi,USSIG,-0.45,MD,Gino Ussi,USSIG,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.27,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,2.55,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.27,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,2.55,RSSD-PS-RM-PURE-HS,Kelechi Okere - RSSD-PS-RM-PURE-HS - 105667,OKEREK - RSSD-PS-RM-PURE-HS - 105667,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1.27,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,2.55,SSM-PS-RM-BPRESS-NOAM-HS,David Stout,STOUTD,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.29,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,2.58,SSM-PS-RM-BPRESS-NOAM-PROFS,Tish Wagner,WAGNERT,0,0,...,0,0,0,0,0,0,0,0,0,0


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = ['TERRITORYNUMBER', 'OWNERNAME', 'ASSIGNTOTERRITORYNUMBER', 'ASSIGNTOTERRITORYOWNERNAME']

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,TERRITORYNAME,OWNERID,ASSIGNTOTERRITORYNAME,ASSIGNTOTERRITORYOWNERID,Books,Engineering,Journals,Licensing,Life Sciences,Other,...,Reaxys,Reaxys Medicinal Chemistry,Research Data Management,SCOPUS,SSRN,SciVal Tools,Scopus Custom Data,bepress,eBooks,embase
0,MD,USSIG,MD,USSIG,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,RSSD-PS-RM-PURE-HS,OKEREK - RSSD-PS-RM-PURE-HS - 105667,RSSD-PS-RM-PURE-HS,OKEREK - RSSD-PS-RM-PURE-HS - 105667,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RSSD-PS-RM-PURE-HS,OKEREK - RSSD-PS-RM-PURE-HS - 105667,RSSD-PS-RM-PURE-HS,OKEREK - RSSD-PS-RM-PURE-HS - 105667,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,SSM-PS-RM-BPRESS-NOAM-HS,STOUTD,SSM-PS-RM-BPRESS-NOAM-HS,STOUTD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SSM-PS-RM-BPRESS-NOAM-PROFS,WAGNERT,SSM-PS-RM-BPRESS-NOAM-PROFS,WAGNERT,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Save clean data to hdf store

In [29]:
store = HDFStore(path_to_clean_hdf_datastore)
store.put(key=cfg['product_assignment_file']+"_clean", value=df)
store.close()
